In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
import warnings
warnings.filterwarnings('ignore')        

In [ ]:
# Определим типы данных для оптимизации памяти
dtype_events = {
    'region': 'category',
    'ua_device_type': 'category',
    'ua_client_type': 'category',
    'ua_os': 'category',
    'ua_client_name': 'category',
    'total_watchtime': 'int32',
    'rutube_video_id': 'category',
    'viewer_uid': 'int32'
}

dtype_video_info = {
    'rutube_video_id': 'category',
    'title': 'object',
    'category': 'category',
    'duration': 'int32',
    'author_id': 'int32'
}

dtype_targets = {
    'viewer_uid': 'int32',
    'gender': 'category',
    'age': 'int8'
}

# Загрузим train_events.csv
train_events = pd.read_csv('/kaggle/input/dadadadadadadadadadadaddadadadadadadadadadadada/train_events.csv', dtype=dtype_events, parse_dates=['event_timestamp'])
print(f"Train Events Shape: {train_events.shape}")

# Загрузим video_info.csv
video_info = pd.read_csv('/kaggle/input/dadadadadadadadadadadaddadadadadadadadadadadada/video_info_v2.csv', dtype=dtype_video_info)
print(f"Video Info Shape: {video_info.shape}")

# Загрузим train_targets.csv
train_targets = pd.read_csv('/kaggle/input/dadadadadadadadadadadaddadadadadadadadadadadada/train_targets.csv', dtype=dtype_targets)
print(f"Train Targets Shape: {train_targets.shape}")

Переведем event_timestamp из GMT+3 под региональное время

In [ ]:
region_timezones = {
    'Chelyabinsk': 'Asia/Yekaterinburg',
    'Bashkortostan Republic': 'Asia/Yekaterinburg',
    'St.-Petersburg': 'Europe/Moscow',
    'Moscow': 'Europe/Moscow',
    'Moscow Oblast': 'Europe/Moscow',
    'Tatarstan Republic': 'Europe/Moscow',
    'Novosibirsk Oblast': 'Asia/Novosibirsk',
    'Omsk Oblast': 'Asia/Omsk',
    'Chuvashia': 'Europe/Moscow',
    'Krasnoyarsk Krai': 'Asia/Krasnoyarsk',
    'Kamchatka': 'Asia/Kamchatka',
    'Nizhny Novgorod Oblast': 'Europe/Moscow',
    'Krasnodar Krai': 'Europe/Moscow',
    'Volgograd Oblast': 'Europe/Moscow',
    'Kaliningrad Oblast': 'Europe/Kaliningrad',
    'Kuzbass': 'Asia/Novosibirsk',
    'Stavropol Kray': 'Europe/Moscow',
    'Samara Oblast': 'Europe/Samara',
    'Amur Oblast': 'Asia/Yakutsk',
    'Sverdlovsk Oblast': 'Asia/Yekaterinburg',
    'Yamalo-Nenets': 'Asia/Yekaterinburg',
    'Orenburg Oblast': 'Asia/Yekaterinburg',
    'Khanty-Mansia': 'Asia/Yekaterinburg',
    'Kaluga Oblast': 'Europe/Moscow',
    'Tomsk Oblast': 'Asia/Novosibirsk',
    'Novgorod Oblast': 'Europe/Moscow',
    'Arkhangelskaya': 'Europe/Moscow',
    'North Ossetia–Alania': 'Europe/Moscow',
    'Kursk Oblast': 'Europe/Moscow',
    "Leningradskaya Oblast'": 'Europe/Moscow',
    'Krasnoyarskiy': 'Asia/Krasnoyarsk',
    'Ivanovo Oblast': 'Europe/Moscow',
    'Altay Kray': 'Asia/Barnaul',
    'Kurgan Oblast': 'Asia/Yekaterinburg',
    'Kostroma Oblast': 'Europe/Moscow',
    'Bryansk Oblast': 'Europe/Moscow',
    'Dagestan': 'Europe/Moscow',
    'Lipetsk Oblast': 'Europe/Moscow',
    'Vladimir Oblast': 'Europe/Moscow',
    'Kirov Oblast': 'Europe/Moscow',
    'Khabarovsk': 'Asia/Khabarovsk',
    'Tambov Oblast': 'Europe/Moscow',
    'Chukotka': 'Asia/Anadyr',
    'Voronezh Oblast': 'Europe/Moscow',
    'Sverdlovsk': 'Asia/Yekaterinburg',
    'Tula Oblast': 'Europe/Moscow',
    'Krasnodarskiy': 'Europe/Moscow',
    'Irkutsk Oblast': 'Asia/Irkutsk',
    'Saratov Oblast': 'Europe/Samara',
    'Khakasiya Republic': 'Asia/Krasnoyarsk',
    'Penza': 'Europe/Moscow',
    'Perm Krai': 'Europe/Yekaterinburg',
    'Oryol oblast': 'Europe/Moscow',
    'Vladimir': 'Europe/Moscow',
    'Smolensk Oblast': 'Europe/Moscow',
    'Penza Oblast': 'Europe/Moscow',
    'Mordoviya Republic': 'Europe/Moscow',
    'Tyumen’ Oblast': 'Asia/Yekaterinburg',
    'Sakha': 'Asia/Yakutsk',
    'Primorye': 'Asia/Vladivostok',
    'Zabaykalskiy (Transbaikal) Kray': 'Asia/Chita',
    'Vologda Oblast': 'Europe/Moscow',
    'Yaroslavl Oblast': 'Europe/Moscow',
    'Crimea': 'Europe/Moscow',
    'Rostov': 'Europe/Moscow',
    'Ryazan Oblast': 'Europe/Moscow',
    'Perm': 'Europe/Yekaterinburg',
    'Chechnya': 'Europe/Moscow',
    'Udmurtiya Republic': 'Asia/Yekaterinburg',
    'Tver Oblast': 'Europe/Moscow',
    'Buryatiya Republic': 'Asia/Ulan-Ude',
    'Belgorod Oblast': 'Europe/Moscow',
    'Kaluga': 'Europe/Moscow',
    'Astrakhan Oblast': 'Europe/Astrakhan',
    'Karelia': 'Europe/Moscow',
    'Murmansk': 'Europe/Moscow',
    'Adygeya Republic': 'Europe/Moscow',
    'Kemerovo Oblast': 'Asia/Novosibirsk',
    'Mariy-El Republic': 'Europe/Moscow',
    'Kursk': 'Europe/Moscow',
    'Saratovskaya Oblast': 'Europe/Samara',
    'Sakhalin Oblast': 'Asia/Vladivostok',
    'Ivanovo': 'Europe/Moscow',
    'Tyumen Oblast': 'Asia/Yekaterinburg',
    'Stavropol’ Kray': 'Europe/Moscow',
    'Voronezj': 'Europe/Moscow',
    'Karachayevo-Cherkesiya Republic': 'Europe/Moscow',
    'Kabardino-Balkariya Republic': 'Europe/Moscow',
    'Ulyanovsk': 'Europe/Moscow',
    'North Ossetia': 'Europe/Moscow',
    'Komi': 'Europe/Moscow',
    'Smolensk': 'Europe/Moscow',
    'Tver’ Oblast': 'Europe/Moscow',
    'Sebastopol City': 'Europe/Moscow',
    'Pskov Oblast': 'Europe/Moscow',
    'Tula': 'Europe/Moscow',
    'Orel Oblast': 'Europe/Moscow',
    'Jaroslavl': 'Europe/Moscow',
    'Tambov': 'Europe/Moscow',
    'Kalmykiya Republic': 'Europe/Moscow',
    'Primorskiy (Maritime) Kray': 'Asia/Vladivostok',
    'Altai': 'Asia/Barnaul',
    'Magadan Oblast': 'Asia/Magadan',
    'Vologda': 'Europe/Moscow',
    'Tyva Republic': 'Asia/Kyzyl',
    'Nenets': 'Europe/Moscow',
    'Smolenskaya Oblast’': 'Europe/Moscow',
    'Jewish Autonomous Oblast': 'Asia/Yakutsk',
    'Astrakhan': 'Europe/Astrakhan',
    'Ingushetiya Republic': 'Europe/Moscow',
    'Kirov': 'Europe/Moscow',
    'Transbaikal Territory': 'Asia/Chita',
    'Omsk': 'Asia/Omsk',
    'Kaliningrad': 'Europe/Kaliningrad',
    'Stavropol Krai': 'Europe/Moscow',
    'Arkhangelsk Oblast': 'Europe/Moscow',
}


from pytz import timezone

def convert_to_local_time(row):
    region = row['region']
    timestamp = row['event_timestamp']
    try:
        # Получаем часовой пояс региона, по умолчанию 'Europe/Moscow'
        tz = timezone(region_timezones.get(region, 'Europe/Moscow'))
    except:
        tz = timezone('Europe/Moscow')
    
    # Проверяем, является ли временная метка timezone-aware
    if timestamp.tzinfo is None or timestamp.tzinfo.utcoffset(timestamp) is None:
        # Если временная метка naive, локализуем её к московскому времени
        timestamp = timestamp.tz_localize('Europe/Moscow')
    
    # Преобразуем время в локальный часовой пояс региона
    return timestamp.astimezone(tz)


train_events['local_event_timestamp'] = train_events.apply(convert_to_local_time, axis=1)

# Объединяем таблицы

Для более удобной работы объединим таблицы train_events, train_targets, video_info

In [ ]:
merged_data = pd.merge(train_events, train_targets, on='viewer_uid', how='left')
merged_data = pd.merge(merged_data, video_info, on='rutube_video_id', how='left')

В итоге у нас получилось 17 признаков.

In [ ]:
merged_data.info()

In [ ]:
merged_data.drop(columns=['event_timestamp'], inplace=True)

# Подготовка данных

In [ ]:
merged_data.describe()

Благодаря описательной статистике можно сделать некоторые выводы

``age``
- Возрастная группа пользователей варьируется от 11 до 54 лет, с большинством пользователей в возрасте около 33 лет.
- Стандартное отклонение (std) = 8.5 года, что говорит о разбросе возрастов пользователей.

``age_class``
- Медиана (50%) = 2. Большинство пользователей находятся в классе "2", что, вероятно, соответствует среднему возрасту.
- Минимум (min): 0, максимум (max): 3. Означает, что есть 4 категории возрастов (0, 1, 2, 3).

``total_watchtime``
- Среднее (mean) равно 2203.43 секунд, что соответствует около 36.72 минут просмотра.
- Стандартное отклонение (std) = 4027.57 секунд (около 67 минут), что говорит о высокой изменчивости времени просмотра.
- Максимальное значение (max) в нашем случае равняется 2,489,070 секунд (~28.8 дней), что может указывать на возможные выбросы (аномальные значения).

### Проверим данные на наличие пропусков (NaN)

In [ ]:
merged_data.isna().sum()

Видим пропуски в одном признаке - ``ua_os``. Надо изучить, в каких случаях появляются пропущенные значения

In [ ]:
merged_data[merged_data['ua_os'].isna()].sample(10)

Как можно заметить, ua_os = NaN в случаях, где тип устройства – smartphone или tablet. Поэтому надо посмотреть, какие значения присутствуют в ua_os, чтобы понять чем можно заполнить пропуски при таких типах устройств.

In [ ]:
merged_data['ua_os'].value_counts()

Большинство устройств работают на Android, поэтому заполним пропуски именно этой ОС.

In [ ]:
# Заполняем NaN в ua_os значением 'Android' с использованием .loc
merged_data.loc[merged_data['ua_os'].isna(), 'ua_os'] = 'Android'

# Проверяем результат, чтобы убедиться, что NaN были заменены
print(merged_data['ua_os'].isna().sum())  # Должно быть 0

Пропуски заполнены

### Проверка на дубликаты

In [ ]:
merged_data.duplicated().sum()

Дубликатов не обнаружено, можно переходить к следующему шагу

### Проверка на наличие выбросов (outliers)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Получение списка числовых признаков, исключая viewer_uid и author_id
numeric_columns = merged_data.select_dtypes(include=['int8', 'int32', 'int64']).columns
numeric_columns = [col for col in numeric_columns if col not in ['viewer_uid', 'author_id']]

# Построение boxplot для каждого числового признака
plt.figure(figsize=(15, 10))  # Размер графиков
for i, col in enumerate(numeric_columns, 1):
    plt.subplot(len(numeric_columns) // 2 + 1, 2, i)  # Сетка для графиков
    sns.boxplot(data=merged_data, x=col)
    plt.title(f'Boxplot for {col}')

plt.tight_layout()  # Автоматическая подгонка
plt.show()

### Графики распределения для признаков с выбросами

In [ ]:
import plotly.express as px

# Построение гистограммы распределения для параметра total_watchtime
fig = px.histogram(merged_data, x='total_watchtime', nbins=50, title='Распределение Total Watchtime')

# Обновление оформления
fig.update_layout(xaxis_title='Total Watchtime', yaxis_title='Count', bargap=0.1)

# Отображение графика
fig.show()

In [ ]:
import plotly.express as px

# Построение гистограммы распределения для параметра total_watchtime
fig = px.histogram(merged_data, x='duration', nbins=50, title='Распределение Duration')

# Обновление оформления
fig.update_layout(xaxis_title='Duration', yaxis_title='Count', bargap=0.1)

# Отображение графика
fig.show()

# Анализ возраста пользователей

Проанализируем более подробнее возраст пользователей платформы RUTUBE

In [ ]:
age_users = merged_data[['viewer_uid', 'age', 'age_class']].drop_duplicates() # Сохраняем только уникальных пользоваетелей с их возрастом и классом

age_counts = age_users['age'].value_counts() # Сохраняем распределение возрастов 

age_counts

Построим столбчатую диаграмму для анализа распределения

In [ ]:
import pandas as pd
import plotly.express as px

# Преобразуем age_counts в DataFrame
age_counts_df = age_counts.reset_index()
age_counts_df.columns = ['age', 'count']  # Переименуем столбцы

# Строим столбчатую диаграмму
fig = px.bar(age_counts_df, x='age', y='count', title='Распределение по возрасту', labels={'age': 'Возраст', 'count': 'Количество'})
fig.update_layout(xaxis_title='Возраст', yaxis_title='Количество', xaxis=dict(tickmode='linear'))

# Показываем график
fig.show()

По этому графику можно сделать несколько интересных выводов.

- Самые популярные "возрасты" – это 24 и 34. Раз данные 2024 года, то можно предположить, что пользователи ставят ненастоящую дату рождения. Оба этих возраста соответствуют круглым датам рождения: 2000 и 1990 годы. Это поведение может быть обусловлено желанием защитить личные данные или просто облегчить процесс регистрации, а также предпочтением круглых дат, которые выглядят более "красиво" и привлекательно.
- Большинство пользователей находятся в диапазоне от 20 до 50 лет, что может свидетельствовать о том, что платформа популярна среди зрелой аудитории. Наибольшее число пользователей наблюдается в возрастах 23–39 лет, что соответствует людям в активном профессиональном возрасте, часто интересующихся контентом на платформах видеостриминга.
- Возрасты ниже 17 лет встречаются крайне редко, что может указывать на то, что такие пользователи чаще указывают возраст старше своего реального, либо на то, что у молодежи нет интереса к платформе.

# Распределение уникальных пользователей по гендерам

In [ ]:
unique_viewer_sex = merged_data[['viewer_uid', 'sex']].drop_duplicates()

sex_counts = unique_viewer_sex['sex'].value_counts()

sex_counts

Далее посмотрим распределение по возрастным группам и гендерам вместе.

In [ ]:
unique_viewer_age_class = merged_data[['viewer_uid', 'age_class', 'sex']].drop_duplicates()

age_class_sex_distribution = unique_viewer_age_class.groupby('sex')['age_class'].value_counts().unstack()

age_class_sex_distribution

Женская аудитория более активна в младших возрастных группах (20–40 лет), а мужская — в старших (30–60 лет).

Для большей наглядности, построим гистограмму.

In [ ]:
import plotly.express as px
import pandas as pd

unique_viewer_age_class = merged_data[['viewer_uid', 'age_class', 'sex']].drop_duplicates()

fig = px.histogram(unique_viewer_age_class, 
                   x='sex', 
                   color='age_class', 
                   barmode='group',
                   title='Распределение возрастных классов по гендерам')

fig.update_layout(xaxis_title='Возрастные классы', yaxis_title='Количество пользователей')

fig.show()

# Анализ категорий видеороликов

In [ ]:
category_videos = merged_data[['rutube_video_id', 'category']].drop_duplicates() # Сохраняем только уникальных пользоваетелей с их возрастом и классом

category_counts = category_videos['category'].value_counts() # Сохраняем распределение возрастов 

# Преобразуем age_counts в DataFrame
category_counts_df = category_counts.reset_index()
category_counts_df.columns = ['category', 'count']  # Переименуем столбцы

# Строим столбчатую диаграмму
fig = px.bar(category_counts_df, x='category', y='count', title='Распределение по возрасту', labels={'category': 'Возраст', 'count': 'Количество'})
fig.update_layout(xaxis_title='Возраст', yaxis_title='Количество', xaxis=dict(tickmode='linear'))

# Показываем график
fig.show()

- Категория "Сериалы" является самой популярной с большим отрывом (32,671), что указывает на высокую заинтересованность пользователей в просмотре серийного контента. 
- Следующей по популярности является категория "Разное" (19,483), что может свидетельствовать о большом объеме контента, который сложно четко классифицировать, либо о том, что пользователи часто ищут разнообразный контент вне четких жанровых рамок.
- Категории "Телепередачи" (18,071) и "Фильмы" (7,736) также занимают важное место, что говорит о том, что пользователи активно смотрят как традиционные программы, так и полнометражные фильмы. Однако телепередачи вдвое более популярны, что может быть связано с их разнообразием и регулярностью выхода.

In [ ]:
category_counts

Далее посмотрим распределение возрастных классов по категориям видеороликов

In [ ]:
unique_viewer_age_class = merged_data[['viewer_uid', 'age_class', 'category']].drop_duplicates()

age_class_distribution = unique_viewer_age_class.groupby('category', observed=True)['age_class'].value_counts().unstack()

age_class_distribution

Опять же, для большей наглядности построим график. Так будет проще провести анализ

In [ ]:
import plotly.express as px
import pandas as pd

unique_viewer_age_class = merged_data[['viewer_uid', 'age_class', 'category']].drop_duplicates()

fig = px.histogram(unique_viewer_age_class, 
                   x='category', 
                   color='age_class', 
                   barmode='group',
                   title='Распределение возрастных классов по категориям')

fig.update_layout(xaxis_title='Категория', yaxis_title='Количество пользователей')

fig.show()

Как можно заметить, большая часть аудитории смотрит "Телепередачи", далее "Сериалы". Самые преобладающие здесь классы – это 1 и 2 (то есть от 20 до 40 лет пользователи)

Теперь посмотрим распределение категорий для каждого пола.

In [ ]:
unique_viewer_age_class = merged_data[['viewer_uid', 'sex', 'category']].drop_duplicates()

age_class_distribution = unique_viewer_age_class.groupby('category', observed=True)['sex'].value_counts().unstack()

age_class_distribution

И построим график для наглядности.

In [ ]:
import plotly.express as px
import pandas as pd

unique_viewer_age_class = merged_data[['viewer_uid', 'sex', 'category']].drop_duplicates()

fig = px.histogram(unique_viewer_age_class, 
                   x='category', 
                   color='sex', 
                   barmode='group',
                   title='Распределение полов по категориям')

fig.update_layout(xaxis_title='Категория', yaxis_title='Количество пользователей')

fig.show()

- Некоторые категории имеют более высокую долю женской аудитории. Например, в категории «Телепередачи» женщин значительно больше (57726 против 24271 мужчин), что может указывать на то, что такие программы пользуются большим интересом у женщин.
- В большинстве категорий наблюдается значительное преобладание мужской аудитории по сравнению с женской. Например, в категориях «Видеоигры» (7365 женщин против 12301 мужчин) и «Спорт» (2444 женщин против 6199 мужчин) разница особенно заметна.
- Популярные категории, такие как «Фильмы», «Сериалы» и «Развлечения», привлекают большое количество зрителей обоих полов, но все же количество мужчин превышает количество женщин.

# Анализ операционных систем пользователей

In [ ]:
merged_data['ua_os'].value_counts()

Из-за большого количества непопулярных ОС было решено перевести в другие категории. Начнем с дистрибутивов Linux.

In [ ]:
linux_distributions = ['GNU/Linux', 'Ubuntu', 'Debian', 'SUSE', 'CentOS', 'Fedora', 'NetBSD', 'OpenBSD']

merged_data['ua_os'] = merged_data['ua_os'].replace(linux_distributions, 'Linux')

merged_data['ua_os'].value_counts()

Теперь когда все дистрибутивы Linux находятся под одной категорией, можно остальные ОС внести в категорию "Other"

In [ ]:
to_replace = ['Fire OS', 'MeeGo', 'BlackBerry OS', 'Windows CE', 'BlackBerry Tablet OS', 'HarmonyOS', 
              'KaiOS', 'Symbian', 'MocorDroid', 'Windows RT', 'Symbian OS Series 60', 'Chrome OS', 'wear os', 'android tv', 'Windows Phone']

merged_data['ua_os'] = merged_data['ua_os'].replace(to_replace, 'Other')

In [ ]:
merged_data['ua_os'].value_counts()

В итоге у нас остается 7 крупных категорий, где преобладает Android.

In [ ]:
import plotly.express as px

# Получение данных с помощью value_counts()
os_counts = merged_data['ua_os'].value_counts()

# Построение круговой диаграммы
fig = px.pie(names=os_counts.index, values=os_counts.values, title='Распределение Операционных систем')

# Отображение графика
fig.show()

- Android занимает лидирующую позицию с 1,248,707 пользователями, значительно превышая другие операционные системы.
- Windows с 398,698 пользователями занимает второе место, но его доля значительно меньше, чем у Android.
- Mac (55,179) и iOS (45,672) имеют сравнительно небольшое представление по сравнению с Android и Windows.
- Остальные операционные системы (GNU/Linux, iPadOS и т.д.) имеют очень низкие показатели, что указывает на ограниченное использование.

# Анализ названий (title) видеороликов

In [ ]:
merged_data['title'].nunique()

В данных более 125 тысяч уникальных названий видеороликов, поэтому возникла интересная теория, которая перетекла в гипотезу.

- Эмодзи или слова, написанные в верхнем регистре, привлекают определенную аудиторию

In [ ]:
import pandas as pd
import re

# Функция для определения наличия эмодзи в названии
def contains_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric shapes extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           "]+", flags=re.UNICODE)
    return bool(emoji_pattern.search(text))

# Функция для определения наличия слов с капсом
def contains_uppercase(text):
    words = text.split()
    return any(word.isupper() for word in words)


# Добавление колонок is_emoji и is_uppercase
merged_data['is_emoji'] = merged_data['title'].apply(contains_emoji)
merged_data['is_uppercase'] = merged_data['title'].apply(contains_uppercase)

In [ ]:
merged_data.sample(10)

Теория заключалась в том, что контент, предназначенный для детей, имеет некие свои особенности в генерации названий видеороликов. Например, видео называется "POPPY PLAYTIME CHAPTER 1😱//СИНЕЕ ЧУЧЕЛО ПОЛНОЕ ПРОХОЖДЕНИЕ 🤣" и оно явно предназначено для младшей аудитории. Можно посмотреть, какие пользователи посмотрели это видео.

In [ ]:
merged_data[merged_data['title'] == 'POPPY PLAYTIME CHAPTER 1😱//СИНЕЕ ЧУЧЕЛО ПОЛНОЕ ПРОХОЖДЕНИЕ 🤣']

Как можно увидеть, это видео посмотрела девушка, 26 лет. Скорее всего, видеоролик смотрел ребенок с аккаунта взрослого человека. Поэтому выдвинутая ранее гипотеза пока не подтверждается.

# Анализ времени суток

In [ ]:
merged_data['local_event_timestamp'].head()

Также возникла идея проанализировать время суток, чтобы увидеть время активности для каждого ``age_class``

Утро (Morning): 06:00 – 12:00  
День (Afternoon): 12:00 – 18:00  
Вечер (Evening): 18:00 – 00:00  
Ночь (Night): 00:00 – 06:00  

In [ ]:
merged_data['local_event_timestamp'] = pd.to_datetime(merged_data['local_event_timestamp'], utc=True)

def get_time_of_day(timestamp):
    hour = timestamp.hour
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

merged_data['time_of_day'] = merged_data['local_event_timestamp'].apply(get_time_of_day)

Результат преобразования timestamp во время суток

In [ ]:
merged_data[['local_event_timestamp', 'time_of_day']].sample(20)

In [ ]:
import plotly.express as px

# Подсчет количества событий по времени суток
time_of_day_counts = merged_data['time_of_day'].value_counts().reset_index()
time_of_day_counts.columns = ['time_of_day', 'count']

# Построение столбчатой диаграммы с разными цветами для каждого времени суток
fig = px.bar(time_of_day_counts, x='time_of_day', y='count', 
             color='time_of_day', 
             color_discrete_map={
                 'Morning': 'lightblue',  # Утро
                 'Afternoon': 'orange',   # День
                 'Evening': 'green',      # Вечер
                 'Night': 'purple'        # Ночь
             },
             labels={'time_of_day': 'Время суток', 'count': 'Количество событий'},
             title='Распределение событий по времени суток')

# Отображение графика
fig.show()

- Дневное время — период пикового потребления контента. Высокий показатель активности в дневные часы может указывать на то, что значительная часть аудитории использует видеоплатформу в рабочие или учебные часы. Это может свидетельствовать либо о том, что люди смотрят контент в перерывах, либо используют платформу как фон для выполнения других задач.
- Утро занимает второе место, что может указывать на поведенческую привычку начинать день с потребления контента, например, во время завтрака или поездки на работу. Такая высокая активность утром может также отражать **важность контента, связанного с новостями или образовательными видео**, которые люди могут смотреть перед началом дня.
- Хотя ночная активность самая низкая, она все равно составляет почти 257 тысяч взаимодействий, что может указывать на наличие активной аудитории с нестандартным графиком (например, люди, работающие в ночные смены, или те, кто живет в других часовых поясах). Это может быть полезным для таргетирования контента в позднее время суток для определенных сегментов аудитории.
- Ожидалось бы, что вечер, как время отдыха, будет пиковым периодом для потребления контента, но данные показывают, что вечерние просмотры ниже, чем дневные и утренние. Это может быть связано с тем, что пользователи в это время больше заняты личными делами, отдыхом или социализацией вне дома, либо переходят на другие формы досуга (например, телевидение или игры).

In [ ]:
import plotly.express as px

# Подсчет количества событий по времени суток и возрастным категориям
time_age_counts = merged_data.groupby(['time_of_day', 'age_class'], observed=False).size().reset_index(name='count')

# Суммирование количества событий по времени суток
total_counts = time_age_counts.groupby('time_of_day')['count'].sum().reset_index()

# Сортировка по возрастанию общего количества событий
sorted_time_of_day = total_counts.sort_values('count')['time_of_day']

# Создание нового столбца с отсортированным порядком
time_age_counts['time_of_day'] = pd.Categorical(time_age_counts['time_of_day'], categories=sorted_time_of_day, ordered=True)

# Построение группированной столбчатой диаграммы
fig = px.bar(time_age_counts, x='time_of_day', y='count', 
             color='age_class', barmode='group',
             color_discrete_map=color_discrete_map,
             labels={'time_of_day': 'Время суток', 'count': 'Количество событий', 'age_class': 'Возрастная категория'},
             title='Распределение событий по времени суток и возрастным категориям')

# Отображение графика
fig.show()

- В течение всех временных интервалов (Afternoon, Evening, Morning, Night) возрастная группа 1 (20-30 лет) и группа 2 (30-40 лет) показывают наибольшее количество взаимодействий. Это может указывать на то, что молодая аудитория активно потребляет контент в любое время дня, что говорит о высокой заинтересованности в видеоформате среди этой возрастной категории.
- Группа 0 (9-20 лет) показывает наименьшую активность в ночное время (7,951). Это может быть связано с режимом сна этой возрастной группы, что говорит о меньшей привлекательности контента для этой аудитории поздно ночью.

# Анализ связей между признаками

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Преобразуем пол в числовую форму (чтобы можно было рассчитать корреляцию)
merged_data['sex_numeric'] = merged_data['sex'].map({'male': 1, 'female': 0})

# Оставляем только числовые столбцы для корреляции
numeric_columns = merged_data[['age', 'total_watchtime', 'age_class', 'duration', 'sex_numeric', 'is_emoji', 'is_uppercase']]

# Вычисляем корреляционную матрицу
correlation_matrix = numeric_columns.corr()

# Построим тепловую карту с использованием seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

# Настроим заголовок и показываем график
plt.title('Correlation Heatmap of Merged Data', fontsize=16)
plt.show()

- Корреляция -0.06 между sex_numeric и total_watchtime говорит о том, что пол может незначительно влиять на общее время просмотра, возможно, указывая на различия в предпочтениях контента между мужчинами и женщинами.
- Корреляция -0.01 между total_watchtime и age близка к нулю, что говорит о слабом или отсутствии значительной зависимости между возрастом и общим временем просмотра. Это может указывать на то, что время, проведенное за просмотром, не зависит от возраста.
- Слабая отрицательная корреляция (-0.10) между sex_numeric и duration указывает на то, что мужчины вероятнее склонны к просмотру более коротких видео по сравнению с женщинами, которые могут предпочитать более длительный контент (например, телепередачи и сериалы).

# Анализ клиентов и устройств

In [ ]:
ua_df = merged_data[['ua_device_type', 'ua_client_type', 'ua_os', 'ua_client_name', 'sex', 'age', 'age_class']]
ua_df.sample(10)

In [ ]:
import plotly.express as px
import pandas as pd

unique_viewer_age_class = merged_data[['viewer_uid', 'sex', 'ua_device_type']].drop_duplicates()

fig = px.histogram(unique_viewer_age_class, 
                   x='ua_device_type', 
                   color='sex', 
                   barmode='group',
                   title='Распределение полов по устройствам')

fig.update_layout(xaxis_title='Категория', yaxis_title='Количество пользователей')

fig.show()

- Женщины чаще используют смартфоны и планшеты.
- Мужчины предпочитают настольные компьютеры, хотя разрыв не столь велик.
- Смартфоны – наиболее популярное устройство среди обоих полов, а планшеты – наименее популярное.

In [ ]:
# Считаем количество каждой категории в ua_client_type
client_type_counts = ua_df['ua_client_type'].value_counts().reset_index()
client_type_counts.columns = ['ua_client_type', 'count']

# Создаем столбчатую диаграмму
fig = px.bar(client_type_counts, 
             x='ua_client_type', 
             y='count', 
             title='Распределение типов клиентов',
             labels={'ua_client_type': 'Тип клиента', 'count': 'Количество'})

# Показываем график
print(client_type_counts)
fig.show()

-  С наибольшим количеством взаимодействий (1,140,259) пользователи предпочитают использовать мобильное приложение Rutube. Это свидетельствует о необходимости оптимизации видео для мобильных устройств и разработки интерактивных функций, которые могут повысить вовлеченность зрителей
- С 619,338 взаимодействиями пользователи также активно используют браузеры (browser) для просмотра видео. Это говорит о том, что веб-версии платформы остаются важными, и нужно уделять внимание их функциональности и пользовательскому интерфейсу, чтобы обеспечить комфортный просмотр на десктопах и мобильных браузерах.
- Всего 19 взаимодействий с браузерами с антивирусами (av) указывают на крайне низкую популярность такого типа клиентских приложений среди пользователей видеохостинга. Это может говорить о недостаточном осведомлении пользователей о преимуществах безопасности или низкой привлекательности таких решений.

Результаты подтверждают тенденцию к росту потребления видео через мобильные приложения, что подчеркивает важность адаптивного дизайна и мобильной оптимизации контента для увеличения времени просмотра и удержания пользователей.

Далее проанализируем клиенты, которыми пользуются зрители Rutube.

In [ ]:
ua_df['ua_client_name'].value_counts()

Rutube как мобильное приложение значительно преобладает по популярности среди пользователей, в то время как браузеры, такие как Yandex Browser (230,202) и Chrome (161,821), показывают заметное вовлечение, что подчеркивает важность обеспечения кросс-платформенной совместимости и оптимизации контента для мобильных устройств и различных браузеров.

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Создаем копию DataFrame, чтобы избежать предупреждений
ua_df_copy = ua_df.copy()

# Преобразуем категориальные столбцы в тип object
ua_df_copy['ua_device_type'] = ua_df_copy['ua_device_type'].astype(str)
ua_df_copy['ua_client_type'] = ua_df_copy['ua_client_type'].astype(str)
ua_df_copy['ua_os'] = ua_df_copy['ua_os'].astype(str)
ua_df_copy['ua_client_name'] = ua_df_copy['ua_client_name'].astype(str)
ua_df_copy['sex'] = ua_df_copy['sex'].astype(str)

# Создаем экземпляр LabelEncoder
label_encoder = LabelEncoder()

# Применяем LabelEncoder к каждому категориальному признаку с использованием .loc
ua_df_copy.loc[:, 'ua_device_type'] = label_encoder.fit_transform(ua_df_copy['ua_device_type'])
ua_df_copy.loc[:, 'ua_client_type'] = label_encoder.fit_transform(ua_df_copy['ua_client_type'])
ua_df_copy.loc[:, 'ua_os'] = label_encoder.fit_transform(ua_df_copy['ua_os'])
ua_df_copy.loc[:, 'ua_client_name'] = label_encoder.fit_transform(ua_df_copy['ua_client_name'])
ua_df_copy.loc[:, 'sex'] = label_encoder.fit_transform(ua_df_copy['sex'])

# Проверяем результат
ua_df_copy.head()

In [ ]:
correlation_matrix = ua_df_copy.corr()

# Построим тепловую карту с использованием seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

# Настроим заголовок и показываем график
plt.title('Correlation Heatmap of Merged Data', fontsize=16)
plt.show()

In [ ]:
correlation_matrix

- Негативные корреляции между sex и ua_device_type (-0.097591) и ua_client_type (-0.128409) могут указывать на то, что пол пользователя немного влияет на выбор устройства и типа клиента. Возможно, мужчины и женщины используют различные устройства и браузеры для доступа к контенту.
- Корреляции между age и ua_device_type (0.010519) и ua_client_type (0.028212) очень слабы, что может означать, что выбор устройства и типа клиента менее зависит от возраста пользователей. Это может указывать на то, что пользователи разных возрастных групп используют схожие устройства для доступа к видеоконтенту.
- Слабая отрицательная корреляция (-0.030141) между age и ua_os также подтверждает, что выбор операционной системы незначительно зависит от возраста. Это может означать, что предпочтения пользователей в выборе операционных систем более предсказуемы по другим факторам, а не по возрасту.